
Para realizar la actividad individual calificable, vamos a modificar el código existente para adaptarlo a la nueva tarea. Aquí están los pasos detallados:

Preparar los datos: Cargar imágenes del estudiante y de fondo.
Entrenar el modelo: Utilizar VGG-16 con transfer learning.
Evaluar el modelo: Realizar predicciones con imágenes no vistas.
Ajustar hiperparámetros: Opcionalmente, ajustar hiperparámetros para mejorar el rendimiento.
Paso 1: Preparar los datos
Primero, cargamos las imágenes del estudiante y de fondo.

In [3]:
import kagglehub

# Download latest version
path_faces = kagglehub.dataset_download("atulanandjha/lfwpeople")
path_animals = kagglehub.dataset_download("alessiocorrado99/animals10")
path_places = kagglehub.dataset_download("shivajbd/imagerotation")



Path to dataset files: /root/.cache/kagglehub/datasets/atulanandjha/lfwpeople/versions/3


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

def load_images(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label)
    return images, labels

# Cargar imágenes del estudiante
student_images, student_labels = load_images('path_to_student_images', 1)

# Cargar imágenes de fondo
background_images, background_labels = load_images('path_to_background_images', 0)

# Combinar y preparar datos
images = np.array(student_images + background_images)
labels = np.array(student_labels + background_labels)
labels = to_categorical(labels)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_student_images'

Paso 2: Entrenar el modelo
Construimos y entrenamos el modelo VGG-16.

In [ ]:
# Definir el callback de EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Cargar la red VGG16 sin las capas superiores
baseModel = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Construir el modelo
model = Sequential()
model.add(baseModel)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Congelar las capas del modelo base
for layer in baseModel.layers:
    layer.trainable = False

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

Paso 3: Evaluar el modelo
Realizar predicciones con imágenes no vistas.

In [ ]:
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    if pred[0][0] > pred[0][1]:
        return 'Fondo'
    else:
        return 'Nombre del estudiante'

# Pruebas
print(predict_image('path_to_new_student_image'))
print(predict_image('path_to_new_background_image'))
print(predict_image('path_to_different_face_image'))

Paso 4: Ajustar hiperparámetros (Opcional)
Puedes ajustar los hiperparámetros como el número de épocas, el tamaño del lote, la función de pérdida y el optimizador para mejorar el rendimiento del modelo.



In [ ]:
# Ejemplo de ajuste de hiperparámetros
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, callbacks=[early_stopping])